Using Tesnorflow/Keras ot build a letter-based LSTM Neural Network.
That predicts the next letter based on the input

In [ ]:
#Importing necessary libaries
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

Importing the corpus/Input

In [ ]:
with open ("Goethe_all.txt", "r") as file:
    text = file.read().replace('\n', '')

Preparing data

In [ ]:
text=text.lower()
len(text)
text[:2000]

"zueignung der morgen kam; es scheuchten seine tritte den leisen schlaf, der mich gelind umfing, daß ich, erwacht, aus meiner stillen hütte den berg hinauf mit frischer seele ging; ich freute mich bei einem jeden schritte der neuen blume, die voll tropfen hing; der junge tag erhob sich mit entzücken, und alles war erquickt, mich zu erquicken. und wie ich stieg, zog von dem fluß der wiesen ein nebel sich in streifen sacht hervor. er wich und wechselte, mich zu umfließen, und wuchs geflügelt mir ums haupt empor: des schönen blicks sollt ich nicht mehr genießen, die gegend deckte mir ein trüber flor; bald sah ich mich von wolken wie umgossen und mit mir selbst in dämmrung eingeschlossen. auf einmal schien die sonne durchzudringen, im nebel ließ sich eine klarheit sehn. hier sank er, leise sich hinabzuschwingen; hier teilt' er steigend sich um wald und höhn. wie hofft ich ihr den ersten gruß zu bringen! sie hofft ich nach der trübe doppelt schön. der luft'ge kampf war lange nicht vollendet

Creating 2 dictionaries to change characters into numbers and vice versa 

In [ ]:
characters = sorted(set(text))

char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

Creating featuers and Label

In [ ]:
SEQ_LENGTH = 60
STEP_SIZE = 3

sentences = []
next_char = []

In [ ]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_char.append(text[i + SEQ_LENGTH])

In [ ]:
x = np.zeros((len(sentences), SEQ_LENGTH,
              len(characters)), dtype=np.bool)
y = np.zeros((len(sentences),
              len(characters)), dtype=np.bool)

for i, satz in enumerate(sentences):
    for t, char in enumerate(satz):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_char[i]]] = 1

Building the LSTM neural network


In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

Training

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(lr=0.01))

model.fit(x, y, batch_size=256, epochs=20)

model.save("Goethe.model")

Epoch 1/20
3483/3483 [==============================] - 73s 21ms/step - loss: 1.7567
Epoch 2/20
3483/3483 [==============================] - 74s 21ms/step - loss: 1.5661
Epoch 3/20
3483/3483 [==============================] - 73s 21ms/step - loss: 1.5247
Epoch 4/20
3483/3483 [==============================] - 73s 21ms/step - loss: 1.4986
Epoch 5/20
3483/3483 [==============================] - 73s 21ms/step - loss: 1.4815
Epoch 6/20
3483/3483 [==============================] - 73s 21ms/step - loss: 1.4699
Epoch 7/20
3483/3483 [==============================] - 73s 21ms/step - loss: 1.4612
Epoch 8/20
3483/3483 [==============================] - 73s 21ms/step - loss: 1.4535
Epoch 9/20
3483/3483 [==============================] - 73s 21ms/step - loss: 1.4479
Epoch 10/20
3483/3483 [==============================] - 73s 21ms/step - loss: 1.4425
Epoch 11/20
3483/3483 [==============================] - 73s 21ms/step - loss: 1.4385
Epoch 12/20
3483/3483 [==============================] - 73s 21

Helper function from Keras to adjust Temperature/Creativity

In [ ]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


Text generating function

In [ ]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions,
                                 temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

Generating text:

In [ ]:
print("0.2",generate_text(140, 0.2))

print("0.4",generate_text(140, 0.4))
print("0.5", generate_text(140, 0.5))
print("0.6",generate_text(140, 0.6))
print("0.7",generate_text(140, 0.7))
print("0,8", generate_text(140, 0.8))

0.2 em andern will's nicht gelten; hüben, drüben muß man schelten den augen zu sehen, das sie sie sich der stadt auf den wegen stand.  das schöne geschäftige schlagen sie den herrn sich den schönen besten
